In [2]:
import tensorflow as tf
import os
from matplotlib import pyplot as plt
import math

# Choosing which GPU this notebook can access
# (useful when running multiple experiments in parallel, on different GPUs):
os.environ["CUDA_VISIBLE_DEVICES"]= "0"

# Some hyper-parameters:
input_shape = [224, 224, 3] # We will resize the input images to this shape
batch_size  = 32            # Images per batch (reduce/increase according to the machine's capability)
num_epochs  = 3             # Max number of training epochs
random_seed = 42            # Seed for some random operations, for reproducibility

In [3]:
import cifar_utils

cifar_info = cifar_utils.get_info()
print(cifar_info)

# Number of classes:
num_classes = cifar_info.features['label'].num_classes

# Number of images:
num_train_imgs = cifar_info.splits['train'].num_examples
num_val_imgs = cifar_info.splits['test'].num_examples

train_steps_per_epoch = math.ceil(num_train_imgs / batch_size)
val_steps_per_epoch   = math.ceil(num_val_imgs / batch_size)

tfds.core.DatasetInfo(
    name='cifar100',
    full_name='cifar100/3.0.2',
    description="""
    This dataset is just like the CIFAR-10, except it has 100 classes containing 600 images each. There are 500 training images and 100 testing images per class. The 100 classes in the CIFAR-100 are grouped into 20 superclasses. Each image comes with a "fine" label (the class to which it belongs) and a "coarse" label (the superclass to which it belongs).
    """,
    homepage='https://www.cs.toronto.edu/~kriz/cifar.html',
    data_dir='/root/tensorflow_datasets/cifar100/3.0.2',
    file_format=tfrecord,
    download_size=160.71 MiB,
    dataset_size=132.03 MiB,
    features=FeaturesDict({
        'coarse_label': ClassLabel(shape=(), dtype=int64, num_classes=20),
        'id': Text(shape=(), dtype=string),
        'image': Image(shape=(32, 32, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=100),
    }),
    supervised_keys=('image', 'label'),
    disable_shuf

In [4]:
train_cifar_dataset = cifar_utils.get_dataset(
    phase='train', batch_size=batch_size, num_epochs=num_epochs, shuffle=True,
    input_shape=input_shape, seed=random_seed)

val_cifar_dataset = cifar_utils.get_dataset(
    phase='test', batch_size=batch_size, num_epochs=1, shuffle=False,
    input_shape=input_shape, seed=random_seed)

Reusing ResNet-50 from Keras Apps
we will instantiate a ResNet-50 model, with **randomly-initialized parameters**


In [5]:
resnet50 = tf.keras.applications.resnet50.ResNet50(
    include_top=True, weights=None,
    input_shape=input_shape, classes=num_classes)
resnet50.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

Training the Network
Once instantiated, our model from Keras Applications is like any ohter Keras model. Exactly as we did previously, we will train it on CIFAR:

In [ ]:
import collections
import functools
from keras_custom_callbacks import SimpleLogCallback

# Defining a custom metrics (top-5 accuracy), from a more generic one provided by Keras:
accuracy_metric = tf.metrics.SparseCategoricalAccuracy(name='acc')
top5_accuracy_metric = tf.metrics.SparseTopKCategoricalAccuracy(k=5, name='top5_acc')

metrics_to_print = collections.OrderedDict([("loss", "loss"),
                                            ("v-loss", "val_loss"),
                                            ("acc", "acc"),
                                            ("v-acc", "val_acc"),
                                            ("top5-acc", "top5_acc"),
                                            ("v-top5-acc", "val_top5_acc")])

# Defining optimizer and callbacks for the training:
optimizer = tf.keras.optimizers.Adam()

model_dir = './models/resnet_keras_app'
callbacks = [
    # Callback to interrupt the training if the validation loss/metrics stops improving for some epochs:
    tf.keras.callbacks.EarlyStopping(patience=8, monitor='val_acc',
                                     restore_best_weights=True),
    # Callback to log the graph, losses and metrics into TensorBoard:
    tf.keras.callbacks.TensorBoard(log_dir=model_dir, histogram_freq=0, write_graph=True),
    # Callback to save the model, e.g., every 5 epochs:
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join(model_dir, 'weights-epoch{epoch:02d}-loss{val_loss:.2f}.h5'), period=5),
    # Callback to simply log metrics at the end of each epoch (saving space compared to verbose=1/2):
    SimpleLogCallback(metrics_to_print, num_epochs=num_epochs, log_frequency=1)
]

# Finally, compiling and training:
resnet50.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy',
                 metrics=[accuracy_metric, top5_accuracy_metric])

history = resnet50.fit(train_cifar_dataset,  epochs=num_epochs, steps_per_epoch=train_steps_per_epoch,
                       validation_data=val_cifar_dataset, validation_steps=val_steps_per_epoch,
                       verbose=1, callbacks=callbacks)

Training: start.
Epoch 1/3
   6/1563 [..............................] - ETA: 7:40 - loss: 7.5422 - acc: 0.0052 - top5_acc: 0.0312    

1563/1563 [==============================] - ETA: 0s - loss: 4.1696 - acc: 0.0728 - top5_acc: 0.2448

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(15,10), sharex='col')
ax[0, 0].set_title("loss")
ax[0, 1].set_title("val-loss")
ax[1, 0].set_title("acc")
ax[1, 1].set_title("val-acc")
ax[2, 0].set_title("top5-acc")
ax[2, 1].set_title("val-top5-acc")

ax[0, 0].plot(history.history['loss'])
ax[0, 1].plot(history.history['val_loss'])
ax[1, 0].plot(history.history['acc'])
ax[1, 1].plot(history.history['val_acc'])
ax[2, 0].plot(history.history['top5_acc'])
ax[2, 1].plot(history.history['val_top5_acc'])

# Visual Analysis of the Model Performance

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import os
from PIL import Image

# Preprocesamiento de las imágenes
def preprocess_image(image, label):
    image = tf.image.resize(image, (input_shape[0], input_shape[1]))
    image = image / 255.0
    people_classes = tf.constant([2, 11, 35, 46, 98], dtype=tf.int64)  # Ensure dtype matches label's dtype
    label = tf.cast(label, tf.int64)  # Explicitly cast label to int64
    is_person = tf.reduce_any(tf.equal(label, people_classes))
    return image, is_person

# Cargar el conjunto de datos CIFAR-100
dataset, info = tfds.load('cifar100', split=['train', 'test'], with_info=True, as_supervised=True)
train_data, test_data = dataset

# Definir la forma de entrada y el tamaño del lote
input_shape = (224, 224, 3)
batch_size = 32

# Aplicar el preprocesamiento
train_dataset = train_data.map(preprocess_image).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
validation_dataset = test_data.map(preprocess_image).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

# Definir el modelo base (ResNet50 preentrenado en CIFAR)
base_model = tf.keras.applications.resnet50.ResNet50(
    include_top=False, weights='imagenet',
    input_shape=input_shape)

# Congelar los pesos del modelo base
for layer in base_model.layers:
    layer.trainable = False

# Añadir nuevas capas al final del modelo
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

# Crear el nuevo modelo
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo en el conjunto de datos de reconocimiento facial
model.fit(train_dataset, epochs=num_epochs)

# Evaluar el modelo en el conjunto de prueba de reconocimiento facial
loss, accuracy = model.evaluate(validation_dataset)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

In [ ]:
import matplotlib.pyplot as plt

# Obtén un lote de imágenes y etiquetas del conjunto de validación
for images, labels in validation_dataset.take(1):
    # Realiza predicciones con el modelo
    predictions = model.predict(images)

# Obtén las etiquetas predichas (el índice de la clase con la mayor probabilidad)
predicted_labels = tf.argmax(predictions, axis=1)

# Muestra las primeras 25 imágenes, sus etiquetas predichas y las etiquetas verdaderas
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(images[i], cmap=plt.cm.binary)
    # Las etiquetas predichas son azules y las etiquetas verdaderas son verdes
    plt.xlabel(f"{predicted_labels[i]}, {labels[i]}")
plt.show()
